In [8]:
import pandas as pd


from sklearn.feature_selection import RFE

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [3]:
df = pd.read_csv("Combined_data.csv")

In [5]:
df.head(3)

,Date,Mean-value,Interest-rate,Month,Year,Quarter,Week-of-year,Week-of-month,Day-of-week,Day-of-year,election-year
0,10/11/2016,101.28,0.69,10.0,2016.0,4.0,42.0,3.0,2.0,285.0,0
1,10/13/2016,101.30,0.66,10.0,2016.0,4.0,42.0,3.0,4.0,287.0,0
2,10/13/2016,101.30,0.66,10.0,2016.0,4.0,42.0,3.0,4.0,287.0,0


In [98]:
X = df.drop(columns = ["Date", "Mean-value", "Year"], axis=1)
y = df["Mean-value"]

#### RECURSIVE FEATURE ELIMINATION

##### Lightgbm model

In [30]:
estimator = LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
selector = RFE(estimator, n_features_to_select=5, step=1)
lgbm_rfe = selector.fit_transform(X, y)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 567
[LightGBM] [Info] Number of data points in the train set: 2078, number of used features: 8
[LightGBM] [Info] Start training from score 114.161442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [32]:
# Get the boolean mask of selected features
selected_mask = selector.support_

# If X is a DataFrame, get the names of the selected features
if hasattr(X, 'columns'):
    selected_features = X.columns[selected_mask]
else:
    # If X is a NumPy array, get the indices of selected features
    selected_features = [f"Feature_{i}" for i, selected in enumerate(selected_mask) if selected]

print("Selected Features:", selected_features)

Selected Features: Index(['Interest-rate', 'Month', 'Week-of-year', 'Day-of-year',
       'election-year'],
      dtype='object')


##### Random Forest Model

In [26]:
estimator = RandomForestRegressor()
rf_selector = RFE(estimator, n_features_to_select=5, step=1)
rf_rfe = rf_selector.fit_transform(X, y)

In [28]:
# Get the boolean mask of selected features
selected_mask = rf_selector.support_

# If X is a DataFrame, get the names of the selected features
if hasattr(X, 'columns'):
    selected_features = X.columns[selected_mask]
else:
    # If X is a NumPy array, get the indices of selected features
    selected_features = [f"Feature_{i}" for i, selected in enumerate(selected_mask) if selected]

print("Selected Features:", selected_features)

Selected Features: Index(['Interest-rate', 'Month', 'Week-of-year', 'Day-of-year',
       'election-year'],
      dtype='object')


In [34]:
X.head(3)

,Interest-rate,Month,Quarter,Week-of-year,Week-of-month,Day-of-week,Day-of-year,election-year
0,0.69,10.0,4.0,42.0,3.0,2.0,285.0,0
1,0.66,10.0,4.0,42.0,3.0,4.0,287.0,0
2,0.66,10.0,4.0,42.0,3.0,4.0,287.0,0


#### Training Random Forest & LightGBM Models using the selected features

In [114]:
X = df[['Interest-rate', 'Month', 'Week-of-year', 'Day-of-year',
       'election-year']]
y = df["Mean-value"]

In [116]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=9)

In [118]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1558, 5), (520, 5), (1558,), (520,))

##### Lightgbm

In [105]:
lgbm = LGBMRegressor(n_estimators=100, learning_rate=1, max_depth=3)

In [107]:
lgbm_model = lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 574
[LightGBM] [Info] Number of data points in the train set: 1558, number of used features: 8
[LightGBM] [Info] Start training from score 113.982750
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [109]:
lgbm_model.score(X_test, y_test)

0.8700239611026261

##### Random Forest

In [112]:
rf = RandomForestRegressor()

In [120]:
rf_model = rf.fit(X_train, y_train)

In [122]:
rf_model.score(X_test, y_test)

0.8750022756463213